In [ ]:
#!/usr/bin/env python3

**This script serves to outline foundamental concepts of the moveit_py motion planning**

In [ ]:
import time
import rclpy
from rclpy.logging import get_logger
#import moveit python library
from moveit.core.robot_state import RobotState
from moveit.planning import (MoveItPy,MultiPipelinePlanRequestParameters)

In [ ]:
failed = True

In [ ]:
def plan_and_execute(
    robot,
    planning_component,
    logger,
    single_plan_parameters=None,
    multi_plan_parameters=None,
    sleep_time=0.0,
):
    """Helper function to plan and execute a motion."""
    # plan to goal
    logger.info("Planning trajectory")
    if multi_plan_parameters is not None:
        plan_result = planning_component.plan(
            multi_plan_parameters=multi_plan_parameters
        )
    elif single_plan_parameters is not None:
        plan_result = planning_component.plan(
            single_plan_parameters=single_plan_parameters
        )
    else:
        plan_result = planning_component.plan()

    # execute the plan
    if plan_result:
        logger.info("Executing plan")
        robot_trajectory = plan_result.trajectory
        robot.execute(robot_trajectory, controllers=[])
    else:
        logger.error("Planning failed")

    time.sleep(sleep_time)


**Here the main function begins, so it is really important to define the steps that are inside the main function**

In [ ]:
def set_states_from_poses(): # this function is an example of how to set start and goal states using predefined poses
    # set start state using predifined state
    lite6_arm.set_start_state(configuration_name="Ready")
    # set goal state using predifined state
    lite6_arm.set_goal_state(configuration_name="Extended") # it does not exist yet!!
    # plan to goal
    plan_and_execute(lite6, lite6_arm, logger, sleep_time=3.0)

In [ ]:
def set_goal_robot_state():
    # instantiate a RobotState instance using the current robot model
    robot_model = lite6.get_robot_model()
    robot_state = RobotState(robot_model)
    # randomize the robot state
    robot_state.set_to_random_positions()
    # set plan start state to current state
    lite6_arm.set_start_state_to_current_state()
    # set goal state to initialized robot state
    logger.info("Set goal state to the initialized robot state")
    lite6_arm.set_goal_state(robot_state=robot_state)
    # plan to goal
    plan_and_execute(lite6, lite6_arm, logger, sleep_time=3.0)
    

In [ ]:
def set_goal_Pose_Stamped(): # ATTENZIONE! " I would suggest using something like numpy-quaternion for making sure your input quaternion is valid and properly normalized, since it's very easy to run into precision issues"
    # Se i valori delle pose non sono ben normalizzati, il comando non diventa valido!
    # set start state to current state
    lite6_arm.set_start_state_to_current_state()

    # set pose goal with PoseStamped messsage
    from geometry_msgs.msg import PoseStamped

    pose_goal = PoseStamped()
    pose_goal.header.frame_id = "link_base"
    pose_goal.pose.orientation.w = 1.0
    pose_goal.pose.position.x = 0.28
    pose_goal.pose.position.y = -0.2
    pose_goal.pose.position.z = 0.5
    lite6_arm.set_goal_state(pose_stamped_msg=pose_goal, pose_link="link_tcp")
    print("aggiornato")

    # plan to goal
    plan_and_execute(lite6, lite6_arm, logger, sleep_time=3.0)

In [ ]:
import numpy as np
from geometry_msgs.msg import PoseStamped, Pose    # set pose goal with PoseStamped messsage

from math import pi

In [ ]:
def get_quaternion_from_euler(roll, pitch, yaw):
  """
  Convert an Euler angle to a quaternion.
   
  Input
    :param roll: The roll (rotation around x-axis) angle in radians.
    :param pitch: The pitch (rotation around y-axis) angle in radians.
    :param yaw: The yaw (rotation around z-axis) angle in radians.
 
  Output
    :return qx, qy, qz, qw: The orientation in quaternion [x,y,z,w] format
  """
  qx = np.sin(roll/2) * np.cos(pitch/2) * np.cos(yaw/2) - np.cos(roll/2) * np.sin(pitch/2) * np.sin(yaw/2)
  qy = np.cos(roll/2) * np.sin(pitch/2) * np.cos(yaw/2) + np.sin(roll/2) * np.cos(pitch/2) * np.sin(yaw/2)
  qz = np.cos(roll/2) * np.cos(pitch/2) * np.sin(yaw/2) - np.sin(roll/2) * np.sin(pitch/2) * np.cos(yaw/2)
  qw = np.cos(roll/2) * np.cos(pitch/2) * np.cos(yaw/2) + np.sin(roll/2) * np.sin(pitch/2) * np.sin(yaw/2)
 
  return [qx, qy, qz, qw]

In [ ]:
print(get_quaternion_from_euler(pi/6,0,pi/2))
good_configurations = []

In [ ]:
def set_goal_from_angles(roll, pitch, yaw): # ATTENZIONE! " I would suggest using something like numpy-quaternion for making sure your input quaternion is valid and properly normalized, since it's very easy to run into precision issues"
    # Se i valori delle pose non sono ben normalizzati, il comando non diventa valido!
    # set start state to current state
    lite6_arm.set_start_state_to_current_state()

    quaternion = get_quaternion_from_euler (roll, pitch, yaw)

    pose_goal = PoseStamped()
    pose_goal.header.frame_id = "link_base"
    pose_goal.pose.orientation.w = quaternion[3]
    pose_goal.pose.position.x = quaternion[0]
    pose_goal.pose.position.y = quaternion[1]
    pose_goal.pose.position.z = quaternion[2]
    lite6_arm.set_goal_state(pose_stamped_msg=pose_goal, pose_link="link_tcp")

    # plan to goal
    plan_and_execute(lite6, lite6_arm, logger, sleep_time=3.0)
    if failed == False:
        print("It has worked")
        # if the the configuration is valid, we store it in the list
        good_configurations.append([roll, pitch, yaw])
    else:
        print("NOT worked")

In [ ]:
print(get_quaternion_from_euler(0,0,-pi/2))
magnitude = np.linalg.norm(get_quaternion_from_euler(0,0,-pi/2))
print("Magnitude:", magnitude)

**l'ipotesi iniziale era che il risultato del braccio dipendesse dalla difficoltà che esso ha nel trovare una soluzione idonea
Sfortunatamene non sembra così, infatti le configurazioni utili del braccio permettono in qualsiasi altra posizione (a quanto ho visto ora) di far muovere il braccio stesso, quindi sono configurazioni apparentemenre sempre valide**

# La roll del braccio deve essere sempre attorno a -pi/2!


In [ ]:
def set_goal_constraints():
    robot_model = lite6.get_robot_model()
    robot_state = RobotState(robot_model)
    # set start state to current state
    lite6_arm.set_start_state_to_current_state()
    # set contraints message
    from moveit.core.kinematic_constraints import construct_joint_constraint


    joint_values = {
        "joint1": -1.0,
        "joint2": 0.7,
        "joint3": 0.7,
        "joint4": -1.5,
        "joint5": -0.7,
        "joint6": 2.0,
    }
    robot_state.joint_positions = joint_values
    joint_constraint = construct_joint_constraint(
        robot_state=robot_state,
        joint_model_group=lite6.get_robot_model().get_joint_model_group("lite6_arm"),
    )
    lite6_arm.set_goal_state(motion_plan_constraints=[joint_constraint])

    # plan to goal
    plan_and_execute(lite6, lite6_arm, logger, sleep_time=3.0)

In [ ]:
# we need to specify our moveit_py config at the top of each notebook we use. 
# this is since we will start spinning a moveit_py node within this notebook.
from moveit_configs_utils import MoveItConfigsBuilder
from ament_index_python.packages import get_package_share_directory

moveit_config = (
        MoveItConfigsBuilder(robot_name="UF_ROBOT", package_name="lite6_enrico")
        .robot_description_semantic(file_path="config/UF_ROBOT.srdf")
        .trajectory_execution(file_path="config/moveit_controllers.yaml")
        .robot_description(file_path="config/UF_ROBOT.urdf.xacro")
        .robot_description_kinematics(file_path="config/kinematics.yaml")
        .joint_limits(file_path="config/joint_limits.yaml")
        .moveit_cpp(
            file_path=get_package_share_directory("lite6_moveit_demos")
            + "/config/moveit_cpp.yaml"
        )
        .to_moveit_configs()
    ).to_dict()







In [ ]:
# initialise rclpy (only for logging purposes)
#rclpy.init()
logger = get_logger("moveit_py.pose_goal")

# instantiate moveit_py instance and a planning component for the panda_arm
lite6 = MoveItPy(node_name="moveit_py", config_dict=moveit_config)
lite6_arm = lite6.get_planning_component("lite6_arm")

# ESEGUI DA QUA A SOPRA!

In [ ]:
planning_scene_monitor = lite6.get_planning_scene_monitor()
with planning_scene_monitor.read_write() as scene:
    robot_model = lite6.get_robot_model()
    robot_state = RobotState(robot_model)
    robot_state.update(force=True)
    print(planning_scene_monitor.name)

In [ ]:
"""
# Initialize rclpy and create an instance of the MoveItPy class
rclpy.init()
logger = get_logger("moveit_py.pose_goal")

# instantiate MoveItPy instance and get planning component
lite6 = MoveItPy(node_name="moveit_py")
lite6_arm = lite6.get_planning_component("lite6_arm")
logger.info("MoveItPy instance created")
"""

In [ ]:
#set_goal_from_angles(-pi/6,0,pi/2)
#set_goal_from_angles(0,0,pi/2) #in this configuration, the gripper is pointing up
#set_goal_from_angles(pi/15,0,pi/2)
set_goal_from_angles(pi/15,-pi/8,pi/2)

#set_goal_from_angles(-3.1415836,0.0499413,0.0000545)

In [ ]:
import random # Funziona, l'unico problema è che trova troppe poche volte una soluzione accettabile, perché?

for __ in range(50):
    robot_model = lite6.get_robot_model()
    robot_state = RobotState(robot_model)
    robot_state.update(force=True)

    
    #roll = random.uniform(0, 1)
    pitch = random.uniform(0, 1)
    yaw = random.uniform(0, 1)
    roll = -pi/2

    magnitude = np.linalg.norm(get_quaternion_from_euler(roll, pitch, yaw))
    print("Magnitude:", magnitude)

    print("roll is", roll,"pitch is", pitch,"yaw is", yaw)
    print("quaternions are", get_quaternion_from_euler(roll, pitch, yaw))
    set_goal_from_angles(roll,pitch,yaw)

    time.sleep(3.0)

In [ ]:
set_states_from_poses() # FUNZIONA, Il robot va più veloce se chiamato attraverso le funzioni fornite, come mai?

In [ ]:
set_goal_Pose_Stamped() # FUNZIONA, attento a settare bene orienamento e posizione però

In [ ]:
set_goal_robot_state() # FUNZIONA, movimento randomico -> sembra che il robopt non si muove in modo randomico, ma è come se scegliesse una soliuzione accettabile e si muovesse sempre in quella direzione.

In [ ]:
set_goal_constraints() #FUNZIONA, stare attento però alla ws in cui deve lavorare il robot

In [ ]:
# KERNEL CRASHA, PERCHÉ? A quanto pare il kernel crasha quando si cerca di settare una posa iniziale e finale con lo stesso stato del robot. Perché?
# Il problema risiede nel fatto che il robot_state.update() non viene chiamato dopo aver settato la posa iniziale, quindi il kernel crasha perché non riesce a trovare una soluzione valida.

# FUNZIONA.

import random

for i in range(30):
    planning_scene_monitor = lite6.get_planning_scene_monitor()
    with planning_scene_monitor.read_write() as scene:
            robot_state = scene.current_state
            original_joint_positions = robot_state.get_joint_group_positions("lite6_arm")

            lite6_arm.set_start_state_to_current_state()

            check_init_pose = robot_state.get_pose("link_tcp")
            print("Initial_pose:", check_init_pose)

            #Set the pose goal, funziona sempre
            pose_goal = Pose()
            pose_goal.position.x = random.uniform(0.3, 0.4) + 0.01*i
            pose_goal.position.y = random.uniform(0.3, 0.4)
            pose_goal.position.z = random.uniform(0.2, 0.4)
            pose_goal.orientation.w = 1.0

            # FUNZIONA, ma una volta sola
            #pose_goal = Pose()
            #pose_goal.position.x = check_init_pose.position.x 
            #pose_goal.position.y = check_init_pose.position.y 
            #pose_goal.position.z = check_init_pose.position.z 
            #pose_goal.orientation.x = check_init_pose.orientation.x 
            #pose_goal.orientation.y = check_init_pose.orientation.y
            #pose_goal.orientation.z = check_init_pose.orientation.z
            #pose_goal.orientation.w = check_init_pose.orientation.w
            #print("Offset_pose:", pose_goal)

            #pose_goal.position.x = 0.1
            #pose_goal.position.y = 0.2
            #pose_goal.position.z = 0.22
            #pose_goal.orientation.w = 1.0
            
            # Set the robot state and check collisions
            result = robot_state.set_from_ik("lite6_arm", pose_goal, "link_tcp", timeout=2.0)
            if not result:
                logger.error("IK solution was not found!")
            else:
                logger.info("IK solution found!")
            
            lite6_arm.set_goal_state(robot_state=robot_state)
            
            robot_state.update() # otherwise the kernel crashes

            check_updated_pose = robot_state.get_pose("link_tcp")

            print("New_pose:", check_updated_pose)

            # set goal state to the initialized robot state
            logger.info("Go to goal")
            #lite6_arm.set_goal_state(robot_state=robot_state)

            #robot_state.update()
            #time.sleep(3.0)
            #plan_and_execute(lite6, lite6_arm, logger,sleep_time=3.0)



            robot_state.set_joint_group_positions(
            "lite6_arm",
            original_joint_positions,
                )
            robot_state.update()

    plan_and_execute(lite6, lite6_arm, logger,sleep_time=1.0) 
        
        

In [ ]:
robot_state.set_joint_group_positions(
        "lite6_arm",
        original_joint_positions,
)
robot_state.update()


In [ ]:
plan_and_execute(lite6, lite6_arm, logger,sleep_time=3.0)

In [ ]:
# Questa cella funziona solo dopo che ho mosso manualmente il robot attraverso rviz, dopodiche funziona. Il problema risiede ad un update sbagliato della planning scene o del robot state.
print("current:", check_init_pose)
robot_state.update()
plan_and_execute(lite6, lite6_arm, logger,sleep_time=3.0)

# Restore the original state
"""
robot_state.set_joint_group_positions(
        "lite6_arm",
        original_joint_positions,
)
"""
#robot_state.update()  # required to update transforms

In [ ]:
planning_scene_monitor = lite6.get_planning_scene_monitor()
with planning_scene_monitor.read_write() as scene:
            # instantiate a RobotState instance using the current robot model
            robot_state = scene.current_state
            original_joint_positions = robot_state.get_joint_group_positions("lite6_arm")
            robot_state.update()

            check_init_pose = robot_state.get_pose("link_tcp")
            print("Initial_pose:", check_init_pose)

            pose_goal = Pose()
            pose_goal.position.x = check_init_pose.position.x
            pose_goal.position.y = check_init_pose.position.y
            pose_goal.position.z = check_init_pose.position.z + 0.1
            pose_goal.orientation.x = check_init_pose.orientation.x
            pose_goal.orientation.y = check_init_pose.orientation.y
            pose_goal.orientation.z = check_init_pose.orientation.z
            pose_goal.orientation.w = check_init_pose.orientation.w
            print("Offset_pose:", pose_goal)

            reached = False

            while reached == False:
                # randomize the robot state
                result = robot_state.set_from_ik(joint_model_group_name="lite6_arm",
                    geometry_pose=pose_goal,
                    tip_name="link_tcp",
                    timeout=2.0)

                # Check the result of the IK solver
                if not result:
                    logger.error("IK solution was not found!")
                else:
                    reached = True

            robot_state.update()

            check_updated_pose = robot_state.get_pose("link_tcp")

            print("New_pose:", check_updated_pose)

            # set plan start state to current state
            lite6_arm.set_start_state_to_current_state()


            # set goal state to the initialized robot state
            logger.info("Go to goal")
            lite6_arm.set_goal_state(robot_state=robot_state)
            robot_state.update()

# plan to goal
plan_and_execute(lite6, lite6_arm, logger,sleep_time=3.0)
                    






In [ ]:
def go_to_pose():
        planning_scene_monitor = lite6.get_planning_scene_monitor()
        with planning_scene_monitor.read_write() as scene:

            # instantiate a RobotState instance using the current robot model
            robot_state = scene.current_state
            original_joint_positions = robot_state.get_joint_group_positions("lite6_arm")
            robot_state.update(force=True)

            check_init_pose = robot_state.get_pose("link_tcp")
            print("Initial_pose:", check_init_pose)

            pose_goal = Pose()
            pose_goal.position.x = check_init_pose.position.x
            pose_goal.position.y = check_init_pose.position.y 
            pose_goal.position.z = check_init_pose.position.z + 0.01 
            pose_goal.orientation.x = check_init_pose.orientation.x
            pose_goal.orientation.y = check_init_pose.orientation.y
            pose_goal.orientation.z = check_init_pose.orientation.z
            pose_goal.orientation.w = check_init_pose.orientation.w
            print("Offset_pose:", pose_goal)


            # randomize the robot state
            result = robot_state.set_from_ik(joint_model_group_name="lite6_arm",
                geometry_pose=pose_goal,
                tip_name="link_tcp",
                timeout=5.0)
            
            # Check the result of the IK solver
            if not result:
                logger.error("IK solution was not found!")
                print("Initial_pose:", check_init_pose)
            else:

                robot_state.update()

                check_updated_pose = robot_state.get_pose("link_tcp")

                print("New_pose:", check_updated_pose)

                # set plan start state to current state
                lite6_arm.set_start_state_to_current_state()


                # set goal state to the initialized robot state
                logger.info("Go to goal")
                lite6_arm.set_goal_state(robot_state=robot_state)
                robot_state.update()

                # plan to goal     
                plan_and_execute(lite6, lite6_arm, logger,sleep_time=3.0)

In [ ]:
go_to_pose() # funziona se parti dallo stato READY, perchè?? Nota, sembra che a dire il vero non funziona proprio, infatti qualsiasi valore metto nella funzione go_to_pose, va sempre nella stessa